INSTALLATION OF PACKAGES 

In [1]:
!pip install screen_brightness_control
#python tool posted on pypi for controlling brightness of primary display
!pip install -U imbalanced-learn
#an open source library relying on scikit-learn to deal with imbalanced classes

IMPORTING MODULES ALREADY PRESENT WITHIN PYTHON 

In [2]:
import time
import datetime 
import random

import ctypes
import pandas as pd
import numpy as np
import screen_brightness_control as sbc

#to access the Windows registry to modify low level functions of the API
import winreg
import subprocess

DISPLAY OF CREATED DATAFRAME with 288382 datapoints having raw gaze coordinates: horizontal(x) and vertical (y)

In [3]:
df=pd.read_csv('dataset1_train_test.csv') #upload whichever dataset you want to out of the 6 and change the filename here
df

,participant,activity,x,y,set,timestamp
0,p01,BROWSE,1199,665,A,199472.0
1,p01,BROWSE,1199,561,A,170889.0
2,p01,BROWSE,1198,553,A,207386.0
3,p01,BROWSE,1197,604,A,188832.0
4,p01,BROWSE,1197,554,A,152954.0
...,...,...,...,...,...,...
288377,p10,BROWSE,576,562,A,208031.0
288378,p10,BROWSE,579,562,A,208068.0
288379,p10,BROWSE,659,562,A,202015.0
288380,p10,BROWSE,663,562,A,199177.0


In [4]:
#DROPPING OFF UNREQUIRED FEATURE COLUMNS
df=df.drop('set',axis=1)
df=df.drop('timestamp',axis=1)

mask=df['participant']=='p10' #cross verify the participant label of the test split by the table provided and change to 'p09' or 'p10' accordingly
Y=df[mask]
Y #display of test split for verification

,participant,activity,x,y
279304,p10,BROWSE,682,797
279305,p10,BROWSE,701,797
279306,p10,BROWSE,572,793
279307,p10,BROWSE,814,793
279308,p10,BROWSE,987,793
...,...,...,...,...
288377,p10,BROWSE,576,562
288378,p10,BROWSE,579,562
288379,p10,BROWSE,659,562
288380,p10,BROWSE,663,562


CONVERSION OF CATEGORICAL DATA INTO NUMERIC DATA

models work more efficiently with numeric values consequently, values of feature column 'participant' are encoded to numeric values

In [5]:
df_encoded = pd.get_dummies(df, columns=['participant'])

CREATION OF FEATURE DATASET AND TARGET DATASET FOR TRAINING OF MODEL

In [6]:
y=df_encoded.activity
X=df_encoded.drop('activity',axis=1)
X

,x,y,participant_p01,participant_p02,participant_p03,participant_p04,participant_p05,participant_p06,participant_p10
0,1199,665,1,0,0,0,0,0,0
1,1199,561,1,0,0,0,0,0,0
2,1198,553,1,0,0,0,0,0,0
3,1197,604,1,0,0,0,0,0,0
4,1197,554,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
288377,576,562,0,0,0,0,0,0,1
288378,579,562,0,0,0,0,0,0,1
288379,659,562,0,0,0,0,0,0,1
288380,663,562,0,0,0,0,0,0,1


IMPORTING LIBRARIES TO USE FOR MACHINE LEARNING IMPLEMENTATION

In [7]:
#allows implementation of said tasks
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

#helps in processing and working with imbalanced classes
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline

In [8]:
#splitting of datasets into training and testing dataset
#choose test_size according to the table given previously by cross-referring the dataset you're using
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.96852092,random_state=0)

DEALING WITH IMBALANCED CLASSES IN THE DATASET BY COMBINATION OF RESAMPLING TECHNIQUES

In [9]:
resampling_pipeline = make_pipeline(SMOTE(sampling_strategy='auto'), RandomUnderSampler(sampling_strategy='auto'))
X_train_resampled, y_train_resampled = resampling_pipeline.fit_resample(X_train, y_train)

FITTING THE DATA TO CHOSEN RANDOM FOREST CLASSIFIER WITH BEST HYPERPARAMETERS FOUND BY GRID-SEARCH AFTER MID-EVAL

In [10]:
rfc=RandomForestClassifier(n_estimators=100, max_features='sqrt', random_state=42)
rfc.fit(X_train_resampled,y_train_resampled)

RandomForestClassifier(random_state=42)

In [11]:
y_pred=rfc.predict(X_test)
y_pred #display of array containing the votes of individual decision trees in the random forest

array(['DEBUG', 'BROWSE', 'DEBUG', ..., 'WATCH', 'PLAY', 'DEBUG'],
      dtype=object)

### IMPLEMENTATION OF DYNAMIC ADAPTATION

Using SBC library that was imported earlier to access display settings of hardware: POWER HUNGRY BRIGHTNESS RESOURCE OPTIMISATION

In [12]:
brightness = sbc.get_brightness()
primary = sbc.get_brightness(display=0)

In [17]:
print("Current Primary display brightness: ",primary)

Current Primary display brightness:  [100]


In [19]:
#changes in brightness will also work according to the time of day, specific settings for daytime and nighttime 
now=datetime.datetime.now().time()

morning_start=datetime.time(7,0)
evening_start=datetime.time(19,0)

Using Windows Registry to access the API's supported battery plans: COMPUTATIONAL RESOURCES MANAGEMENT AND OPTIMISATION

In [13]:
def get_supported_battery_plans():
    battery_plans = {}

    with winreg.OpenKey(winreg.HKEY_LOCAL_MACHINE, r"SYSTEM\CurrentControlSet\Control\Power\User\PowerSchemes") as key:
        subkey_index = 0
        while True:
            try:
                subkey_name = winreg.EnumKey(key, subkey_index)
                subkey_path = rf"SYSTEM\CurrentControlSet\Control\Power\User\PowerSchemes\{subkey_name}"

                with winreg.OpenKey(winreg.HKEY_LOCAL_MACHINE, subkey_path) as subkey:
                    friendly_name, _ = winreg.QueryValueEx(subkey, "FriendlyName")
                    
                    #extracting just the battery plan name from the friendly name
                plan_name = friendly_name.split(",")[-1].strip()
                battery_plans[subkey_name] = plan_name
                subkey_index += 1

            except FileNotFoundError:
                break
            except Exception as e:
                break

    return battery_plans

In [14]:
def set_active_power_plan(plan_id):
    subprocess.run(["powercfg", "/setactive", plan_id])
    supported_battery_plans = get_supported_battery_plans()
    friendly_name= supported_battery_plans[plan_id]
    return friendly_name

Accessing Windows Graphical Device Interface library GDI32 to change colour hues for user-comfort

In [15]:
def set_gamma_ramp(gamma_ramp):
    hdc = ctypes.windll.user32.GetDC(0)
    gamma_ramp_flat = np.require(gamma_ramp, dtype=np.uint16, requirements='C')

    ctypes.windll.gdi32.SetDeviceGammaRamp(hdc, gamma_ramp_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_ushort)))
    ctypes.windll.user32.ReleaseDC(0, hdc)

In [16]:
def generate_hue_gamma_ramp(hue_factor):
    gamma_ramp_size = 256
    gamma_ramp = np.zeros((3, gamma_ramp_size), dtype=np.uint16)

    for i in range(gamma_ramp_size):
        intensity = i / (gamma_ramp_size - 1)
        adjusted_intensity = intensity ** hue_factor
        gamma_ramp[0, i] = gamma_ramp[1, i] = gamma_ramp[2, i] = int(adjusted_intensity * 65535)

    return gamma_ramp

In [18]:
def hue_factor(value,duration_seconds, steps):
    constant_hue_factor = value
    gamma_ramp = generate_hue_gamma_ramp(constant_hue_factor)

    #for smooth transition in colour hues so as to not distress the user
    for step in range(steps):
        set_gamma_ramp(gamma_ramp)
        time.sleep(duration_seconds / steps)

#### CLASS CONTAINING METHODS THAT DEFINE ADAPTIVE FUNCTIONS BASED ON TASK BEING PERFORMED

In [20]:
class ActivityDecider:
       
    @classmethod
    def activity_tap(cls):
        print(f'Majority Vote: {majority_vote}')
    
    @classmethod    
    def BROWSE(cls):
        print("BROWSE MODE ENABLED")
        
        value=set_active_power_plan('381b4222-f694-41f0-9685-ff5bb260df2e')
        print(f"Current Battery plan: {value}")
        
        if morning_start<=now<evening_start:
            sbc.set_brightness(70, display=0)
        else:
            sbc.set_brightness(50, display=0)
            
        hue_factor(1.0,10, 100)  
        print("Shifted to a normal tone.")
        
        
    @classmethod
    def READ(cls):
        print("READ MODE ENABLED")
        
        value=set_active_power_plan('961cc777-2547-4f9d-8174-7d86181b8a7a')
        print(f"Current Battery plan: {value}")
        
        if morning_start<=now<evening_start:
            sbc.set_brightness(55, display=0)
        else:
            sbc.set_brightness(35, display=0)
            
        hue_factor(0.8,10, 100) 
        print("Shifted to a warmer tone.")
        
        
    
    @classmethod
    def PLAY(cls):
        print("PLAY MODE ENABLED")
        
        value=set_active_power_plan('e9a42b02-d5df-448d-aa00-03f14749eb61')
        print(f"Current Battery plan: {value}")
        
        if morning_start<=now<evening_start:
            sbc.set_brightness(100, display=0)
        else:
            sbc.set_brightness(80, display=0)
            
        hue_factor(1.0,10, 100)
        print("Shifted to a normal tone.")
        
        
    @classmethod
    def DEBUG(cls):
        print("DEBUG MODE ENABLED")
        
        value=set_active_power_plan('a1841308-3541-4fab-bc81-f71556f20b4a')
        print(f"Current Battery plan: {value}")
        
        if morning_start<=now<evening_start:
            sbc.set_brightness(45, display=0)
        else:
            sbc.set_brightness(30, display=0)
            
        hue_factor(1.2,10, 100)
        print("Shifted to a cooler tone.")
        
        
    @classmethod
    def WRITE(cls):
        print("WRITE MODE ENABLED")
        
        value=set_active_power_plan('a1841308-3541-4fab-bc81-f71556f20b4a')
        print(f"Current Battery plan: {value}")
        
        if morning_start<=now<evening_start:
            sbc.set_brightness(50, display=0)
        else:
            sbc.set_brightness(40, display=0)
            
        hue_factor(0.8,10, 100)
        print("Shifted to a warmer tone.")
        
    
    @classmethod
    def WATCH(cls):
        print("WATCH MODE ENABLED")
        
        value=set_active_power_plan('381b4222-f694-41f0-9685-ff5bb260df2e')
        print(f"Current Battery plan: {value}")
        
        if morning_start<=now<evening_start:
            sbc.set_brightness(100, display=0)
        else:
            sbc.set_brightness(80, display=0)
            
        hue_factor(1.0,10, 100)
        print("Shifted to a normal tone.")

#### ACCESSING THE MAJORITY CLASS PREDICTED BY THE ML MODEL

In [21]:
majority_vote=max(set(y_pred), key=list(y_pred).count)

DISPLAY LOG OF RESULT FOR CHANGES MADE BY THE PROGRAM

In [22]:
ActivityDecider.activity_tap()
getattr(ActivityDecider, majority_vote)()
for monitor in sbc.list_monitors():
    print(monitor, ':', sbc.get_brightness(display=monitor), '%')

Majority Vote: BROWSE
BROWSE MODE ENABLED
Current Battery plan: Balanced
Shifted to a normal tone.
Asus 0701 : [100] %
